In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/SAFT_ML`


In [37]:
using CSV, DataFrames, DelimitedFiles

In [38]:
df = CSV.read("SI_pcp-saft_parameters.csv",DataFrame,header=1) ;
fieldnames(typeof(df))
display(names(df)[1:20])

writedlm("out.txt", names(df), ' ')


20-element Vector{String}:
 "common_name"
 "iupac_name"
 "inchi"
 "canonical_smiles"
 "isomeric_smiles"
 "cas"
 "family"
 "molarweight"
 "m"
 "sigma"
 "epsilon_k"
 "mu"
 "kappa_ab"
 "epsilon_k_ab"
 "na"
 "nb"
 "mard_psat"
 "t_min_psat"
 "t_max_psat"
 "points_psat"

In [41]:
# Info for Clapeyron Database File:
# species Mw segment sigma epsilon dipole n_H n_e source
num_rows = nrow(df)
df2 = DataFrame(
    species = fill(missing, num_rows),
    Mw = fill(missing, num_rows),
    segment = fill(missing, num_rows),
    sigma = fill(missing, num_rows),
    epsilon = fill(missing, num_rows),
    dipole = fill(missing, num_rows),
    n_H = fill(missing, num_rows),
    n_e = fill(missing, num_rows),
    source = fill(missing, num_rows)

);

In [47]:
df2.species = df.common_name
df2.Mw = df.molarweight
df2.segment = df.m
df2.sigma = df.sigma
df2.epsilon = df.epsilon_k
df2.dipole = df.mu
df2[!, :source] .= "10.1021/acs.iecr.3c02255"
df2

Row,species,Mw,segment,sigma,epsilon,dipole,n_H,n_e,source
,String,Float64,Float64,Float64,Float64,Float64?,Missing,Missing,String
1,acetaldehyde,44.026,2.14028,3.22662,227.621,2.69,missing,missing,10.1021/acs.iecr.3c02255
2,acetamide,59.037,2.82648,3.12045,377.672,missing,missing,missing,10.1021/acs.iecr.3c02255
3,acetonitrile,41.027,2.32567,3.16425,207.704,3.92,missing,missing,10.1021/acs.iecr.3c02255
4,acetone,58.042,2.78618,3.24884,231.118,2.88,missing,missing,10.1021/acs.iecr.3c02255
5,ethylenediamine,60.069,3.97041,2.82526,251.777,missing,missing,missing,10.1021/acs.iecr.3c02255
6,"1,2-dibromoethane",185.868,2.59098,3.5717,323.703,1.0,missing,missing,10.1021/acs.iecr.3c02255
7,ethyl bromide,107.957,2.16801,3.52926,266.03,2.03,missing,missing,10.1021/acs.iecr.3c02255
8,"1,2-ethanediol",62.037,1.71084,3.73481,354.183,missing,missing,missing,10.1021/acs.iecr.3c02255
9,ethyl iodide,155.944,1.94206,3.82267,319.639,1.91,missing,missing,10.1021/acs.iecr.3c02255
